## Dataset Audios creation

##### Randomi MIDI file creation

In [8]:
import random
from basic_pitch import ICASSP_2022_MODEL_PATH, inference

In [20]:
def create_individual(effects, effect_structure):
    n_effects_chosen = random.randint(1, len(effects))
    selected_effects = random.sample(effects, n_effects_chosen)
    
    individ = {}
    for effect in selected_effects:
        if effect in effect_structure:
            structure = effect_structure[effect]
            individ[effect] = {
                param: round(random.uniform(range_[0], range_[1]), 2) 
                for param, (_, range_) in structure.items()
            }
    return individ

In [19]:
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (0.0, 100.0)), },# Chorus
    1: { "delay_seconds": ('float', (0.0, 10.0)), },# Delay
    2: { "drive_db": ('float', (0.0, 50.0)), },# Distortion
    3: { "gain_db": ('float', (-50.0, 50.0)) },# Gain
    4: { "depth": ('float', (0.0, 1.0)), },# Phaser
    5: { "wet_level": ('float', (0.0, 1.0)), },# Reverb
}

In [10]:
create_individual(effects, effect_structure)

{0: {'rate_hz': 61.36}, 3: {'gain_db': 21.29}, 1: {'delay_seconds': 4.48}}

Testing similarity between 2 individuals

In [10]:
import random
import pretty_midi

def generate_random_instrument_midi(filename, program, min_duration, max_duration, min_note, max_note):
    # Create a PrettyMIDI object
    midi_data = pretty_midi.PrettyMIDI()
    
    # Create an Instrument instance for a distortion instrument
    instrument = pretty_midi.Instrument(program=program)
    
    # Determine the length of the MIDI in seconds
    duration = random.randint(min_duration, max_duration)
    min_note_number = pretty_midi.note_name_to_number(min_note)
    max_note_number = pretty_midi.note_name_to_number(max_note)
    
    # Generate random notes
    current_time = 0.0
    while current_time < duration:
        note_number = random.randint(min_note_number, max_note_number)
        
        # Random note duration between 0.1 and 1.0 seconds
        note_duration = random.uniform(0.1, 2.0)
        
        # Ensure that the note ends before the total duration
        note_end_time = min(current_time + note_duration, duration)
        
        # Create a Note instance and add it to the instrument instrument
        note = pretty_midi.Note(
            velocity=random.randint(60, 127),  # Random velocity
            pitch=note_number,
            start=current_time,
            end=note_end_time
        )
        instrument.notes.append(note)
        
        # Update the current time
        current_time = note_end_time
    
    # Add the instrument instrument to the PrettyMIDI object
    midi_data.instruments.append(instrument)
    
    # Write out the MIDI data to a file
    midi_data.write(filename)
    
    return midi_data


In [81]:
midi_file = 'output.mid'
program = 30
min_note = 'E2'
max_note = 'E6' # Generate a random note between MIDI note 40 (E2) and 88 (E6) so basically all notes of the guitar
min_duration = 1
max_duration = 20

midi_data = generate_random_instrument_midi(midi_file, program, min_duration, max_duration, min_note, max_note)

# Using dir() to list all properties and methods
print(dir(midi_data))

# Using __dict__ to see instance attributes
print(midi_data.__dict__)

# Accessing some specific properties
print("Tempo changes:", midi_data.get_tempo_changes())
print("Time signature changes:", midi_data.time_signature_changes)
print("Instruments:", midi_data.instruments[0].notes)
print(midi_data)


['_PrettyMIDI__tick_to_time', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_load_instruments', '_load_metadata', '_load_tempo_changes', '_tick_scales', '_update_tick_to_time', 'adjust_times', 'estimate_beat_start', 'estimate_tempi', 'estimate_tempo', 'fluidsynth', 'get_beats', 'get_chroma', 'get_downbeats', 'get_end_time', 'get_onsets', 'get_piano_roll', 'get_pitch_class_histogram', 'get_pitch_class_transition_matrix', 'get_tempo_changes', 'instruments', 'key_signature_changes', 'lyrics', 'remove_invalid_notes', 'resolution', 'synthesize', 'text_events', 'tick_to_time', 'time_signature_changes', 'time_to_tick', 'write']
{'resolution': 220, '_tick_scales': [(0, 0.0022727272727

In [14]:
from pedalboard import Pedalboard, Reverb, Chorus, Distortion, Delay, Phaser, Compressor, Gain, Clipping
from pedalboard.io import AudioFile

def create_effected_audio(board, file_path, output_file):
    with AudioFile(file_path) as f:
        with AudioFile(output_file, 'w', f.samplerate, f.num_channels) as o:
            while f.tell() < f.frames:
                chunk = f.read(f.samplerate)
                effected = board(chunk, f.samplerate, reset=False)
                o.write(effected)

##### MP3 creation of the MIDI file

In [15]:
from midi2audio import FluidSynth 
from pydub import AudioSegment

def midi_to_mp3(midi_file, mp3_file, soundfont):
    #convert MIDI to WAV using FluidSynth
    fs = FluidSynth(soundfont)
    wav_file = midi_file.replace('.midi', '.wav').replace('.mid', '.wav')
    fs.midi_to_audio(midi_file, wav_file)

    #convert WAV to MP3 using pydub
    sound = AudioSegment.from_wav(wav_file)
    sound.export(mp3_file, format="mp3")

In [22]:
midi_file = "output.mid"
mp3_file = "output.mp3"
soundfont = '../audio2midi2audio/FluidR3_GM.sf2'  # Path to your SoundFont file

midi_to_mp3(midi_file, mp3_file, soundfont)

Conversion complete: output.mp3


##### From MIDI back to MP3

In [21]:
def mp3_to_midi(audio_path, midi_path):
    _, midi_data, __ = inference.predict(
        audio_path,    
        model_or_model_path = ICASSP_2022_MODEL_PATH, 
        onset_threshold = 0.6, #note segmentation 1) how easily a note should be split into two. (split notes <- ..0.5.. -> merge notes)
        frame_threshold = 0.6, #model confidence threshold 2) the model confidence required to create a note. (more notes <- ..0.3.. -> fewer notes)
    )

    for instrument in midi_data.instruments:
        instrument.program = 30 #distortion guitar program

    midi_data.write(midi_path)
    
    return midi_data

## DT creation

In [21]:
import pandas as pd

def audios_dataset_creation(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map):
    for i in range(n_data):
        generate_random_instrument_midi(midi_file, program, min_duration, max_duration, min_note, max_note)
        midi_to_mp3(midi_file, mp3_file, soundfont)
        individual = create_individual(effects, effect_structure)
        board = Pedalboard([])
        for effect_key, params in individual.items():
            effect_class = globals()[effects_map[effect_key]]
            board.append(effect_class(**params))
            
        output_file = str(i) + ".mp3"
        create_effected_audio(board, mp3_file, "audios/" + output_file)
        data["audio_name"].append(output_file)
        data["applied_effects"].append(individual)
        data["GA_effects"].append("")
    
    # Once all the data is collected, you can create the DataFrame
    df = pd.DataFrame(data)

    # Save the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)
    print(f"CSV file '{csv_filename}' created successfully.")

Small test

In [29]:
individ= {2: {'drive_db': 8.06},
 1: {'delay_seconds': 2.52},
 7: {'threshold_db': -0.97},
 3: {'gain_db': -7.94}}
board = Pedalboard([])
for effect_key, params in individ.items():
    effect_class = globals()[effects_map[effect_key]]
    board.append(effect_class(**params))
            
print(board)
output_file = "G4_guitar_TEST.mp3"
create_effected_audio(board, '../audios/G4_guitar.mp3', output_file)

<Pedalboard with 4 plugins: [<pedalboard.Distortion drive_db=8.06 at 00000208D31CB0D0>, <pedalboard.Delay delay_seconds=2.52 feedback=0 mix=0.5 at 00000208D30B9AA0>, <pedalboard.Clipping threshold_db=-0.97 at 00000208D32AC400>, <pedalboard.Gain gain_db=-7.94 at 00000208D31B6DD0>]>


In [22]:
data = {
    "audio_name": [],
    "or_midi": [], 
    "gen_midi": []
}

n_data = 500
midi_file = 'output.mid'
mp3_file = 'output.mp3'
csv_filename = "../results/dataset_audios2.csv"
soundfont = '../audio2midi2audio/FluidR3_GM.sf2'  # Path to your SoundFont file
program = 30
min_note = 'E2'
max_note = 'E6' # Generate a random note between MIDI note 40 (E2) and 88 (E6)
min_duration = 3
max_duration = 10
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (0.0, 100.0)), },# Chorus
    1: { "delay_seconds": ('float', (0.0, 10.0)), },# Delay
    2: { "drive_db": ('float', (0.0, 50.0)), },# Distortion
    3: { "gain_db": ('float', (-50.0, 50.0)) },# Gain
    4: { "depth": ('float', (0.0, 1.0)), },# Phaser
    5: { "wet_level": ('float', (0.0, 1.0)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

In [ ]:
audios_dataset_creation(data, n_data, midi_file, mp3_file, csv_filename, soundfont, program, min_note, max_note, min_duration, max_duration, effects, effect_structure, effects_map)